In [1]:
import pennylane as qml
from pennylane import numpy as np
import qemzmsepcV2new as qem

In [2]:
n_qubits = 4
dev = qml.device('default.mixed', wires=n_qubits)
nqubitschannel = qem.NqubitsChannel(n_qubits)
pauli_channel = nqubitschannel.nqubitsrandompaulichannel(p_identity=0.5)
qemzmsepc = qem.QEMZMSEPC(n_qubits)
p = 0.8

In [3]:
def trotterstep1(rotation_angle_of_rx, rotation_angle_of_rz):
    qml.RX(rotation_angle_of_rx, wires=0)
    qml.RX(rotation_angle_of_rx, wires=1)
    qml.RX(rotation_angle_of_rx, wires=2)
    qml.RX(rotation_angle_of_rx, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.RZ(rotation_angle_of_rz, wires=1)
    qml.RZ(rotation_angle_of_rz, wires=3)
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[2, 3])
    qml.CNOT(wires=[1, 2])
    qml.RZ(rotation_angle_of_rz, wires=2)
    qml.CNOT(wires=[1, 2])

In [4]:
rotation_angles = [0]
rotation_angle_of_rz = -0.2
for rotation_angle_of_rx in rotation_angles:
    z_ideal = qml.QNode(qemzmsepc.noise_circuit, dev)(trotterstep1, rotation_angle_of_rx, rotation_angle_of_rz)
    print(f"z_ideal is {z_ideal}")
    
    z_unmitigated = qml.QNode(qemzmsepc.noise_circuit, dev)(trotterstep1, 
                    rotation_angle_of_rx, rotation_angle_of_rz, p=p, 
                    kraus_matrices_of_a_pauli_channel=pauli_channel, need_gate_noise=True,
                    need_measurement_noise=True)
    print(f"z_unmitigated is {z_unmitigated}")
    
    z_mitigated, _  = qemzmsepc.qemzmsepc(trotterstep1, z_unmitigated, p, dev,
                    rotation_angle_of_rx, rotation_angle_of_rz,
                    kraus_matrices_of_a_pauli_channel=pauli_channel)
    print(f"z_mitigated is {z_mitigated}")
    print("------------------")

z_ideal is 1.0
z_unmitigated is 0.7575810353170114
z_mitigated is 1.000000000000006
------------------
